### 직방의 원룸 매물정보 수집
- 절차
    - 동이름 > 위도,경도 수집
    - 위도,경도 > geohash 변환
    - geohash(영역) > 매물 아이디
    - 매물 아이디 > 매물 정보

In [5]:
import requests
import pandas as pd
import geohash2

In [4]:
!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=ad92bff21e7ce1452d9fbe40ed24e76727515349ed3c25e7fec8124360b353fd
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [ ]:
# 1. 동이름 > 위도, 경도

In [113]:
addr = "강남구 역삼동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

In [14]:
# 2. 위도, 경도 > geohash

In [138]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm6'

In [ ]:
# 3. geohash > 매물 아이디

In [115]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&\
geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [120]:
items = response.json()['items']
items

[{'lat': 37.48566037375152, 'lng': 127.01360661095136, 'item_id': 32819635},
 {'lat': 37.48565886610531, 'lng': 127.0129733743141, 'item_id': 32838798},
 {'lat': 37.48516763871984, 'lng': 127.01296520586361, 'item_id': 32872608},
 {'lat': 37.485629528681926, 'lng': 127.00225179400057, 'item_id': 32838700},
 {'lat': 37.48515172009493, 'lng': 127.03055995657246, 'item_id': 32767020},
 {'lat': 37.48569669534846, 'lng': 127.01239477729017, 'item_id': 32830406},
 {'lat': 37.485691123337155, 'lng': 127.01235381828486, 'item_id': 32862728},
 {'lat': 37.48570863283032, 'lng': 127.00201439365814, 'item_id': 32866406},
 {'lat': 37.485221652800625, 'lng': 127.0131987782606, 'item_id': 32432677},
 {'lat': 37.485253299769695, 'lng': 127.01314911973503, 'item_id': 32578425},
 {'lat': 37.48572803058556, 'lng': 127.01317603446091, 'item_id': 32822492},
 {'lat': 37.48576577683315, 'lng': 127.01454502550781, 'item_id': 32692095},
 {'lat': 37.485298769615945, 'lng': 127.01455515374978, 'item_id': 3279083

In [121]:
items = response.json()["items"]
len(items), items[:2]

(2340,
 [{'lat': 37.48566037375152, 'lng': 127.01360661095136, 'item_id': 32819635},
  {'lat': 37.48565886610531, 'lng': 127.0129733743141, 'item_id': 32838798}])

In [122]:
ids = [item["item_id"] for item in items]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [2]:
# 4. 매물 아이디 > 매물 정보

In [123]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids[:900]
}
response = requests.post(url, params)
response

<Response [200]>

In [124]:
response.text[:200]

'{"items":[{"section_type":null,"item_id":32819635,"images_thumbnail":"https://ic.zigbang.com/ic/items/32819635/1.jpg","sales_type":"전세","sales_title":"전세","deposit":16500,"rent":0,"size_m2":39.89,"공급면'

In [125]:
# max row, max column 설정
pd.options.display.max_columns = 50

In [126]:
items = response.json()["items"]
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
898,32795463,월세,15000,70,69.42,서울시 강남구 역삼동,12
899,32583346,월세,150,120,56.20,서울시 서초구 서초동,10


In [145]:
# 함수 만들기
def oneroom(addr):
    # 위도, 경도 가져옴
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    # 영역 구분
    geohash = geohash2.encode(lat, lng, precision=5)
    # 영역 안 매물 탐색
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&\
geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    # 매물아이디 리스트 만들기
    items = response.json()['items']
    ids = [item["item_id"] for item in items]
    # 매물아이디로 매물조회 요청하기
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    # 지정칼럼에 대해서만 리스트 만들기
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]

In [143]:
addr = "마포구 합정동"
df = oneroom(addr)
df

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
1,32429092,전세,37000,0,39.67,서울시 마포구 합정동,5
2,32526788,전세,36000,0,35.53,서울시 마포구 합정동,4
3,32640863,전세,39000,0,52.89,서울시 마포구 합정동,8
4,32669330,전세,37000,0,35.53,서울시 마포구 합정동,8
5,32669563,전세,37000,0,35.69,서울시 마포구 합정동,8
6,32688900,전세,36500,0,46.94,서울시 마포구 합정동,8
7,32693015,전세,36500,0,35.40,서울시 마포구 합정동,8
8,32697937,전세,37000,0,36.00,서울시 마포구 합정동,8
9,32699400,전세,37500,0,46.28,서울시 마포구 합정동,8


In [133]:
df.filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)

df.filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
93,32562277,월세,1000,90,62.00,서울시 마포구 합정동,2
94,32769021,월세,1000,90,62.81,서울시 마포구 합정동,2
95,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
96,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [ ]:
# 모듈 파일 만들기 : .py 

In [150]:
%%writefile zigbang.py
import requests
import pandas as pd
import geohash2

# 함수 만들기
def oneroom(addr):
    # 위도, 경도 가져옴
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    # 영역 구분
    geohash = geohash2.encode(lat, lng, precision=5)
    # 영역 안 매물 탐색
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&\
geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    # 매물아이디 리스트 만들기
    items = response.json()['items']
    ids = [item["item_id"] for item in items]
    # 매물아이디로 매물조회 요청하기
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    # 지정칼럼에 대해서만 리스트 만들기
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]

Overwriting zigbang.py


In [147]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7C6C-FC35

 C:\Users\User\python라이브러리\day2 디렉터리

2022-08-04  오후 01:33    <DIR>          .
2022-08-04  오후 01:33    <DIR>          ..
2022-08-04  오전 11:08    <DIR>          .ipynb_checkpoints
2022-08-04  오전 11:10            54,228 001_naver_api_2.ipynb
2022-08-04  오후 01:33           130,705 002_zigbang.ipynb
2022-07-22  오전 10:09            20,775 01_html.ipynb
2022-07-10  오전 06:05             7,617 02_css_selector.ipynb
2022-07-10  오전 06:06            21,109 03_naver_relational_keywords.ipynb
2022-08-03  오후 04:33           250,909 04_gmarket.ipynb
2022-08-04  오후 01:33             1,283 zigbang.py
               7개 파일             486,626 바이트
               3개 디렉터리  123,541,729,280 바이트 남음


In [148]:
import zigbang as zb

In [149]:
df = zb.oneroom('망원동')
df

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32864713,전세,9000,0,21.49,서울시 영등포구 당산동4가,13
1,32694522,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
2,32822043,전세,9000,0,16.53,서울시 영등포구 당산동4가,13
3,32827028,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
4,32856780,전세,7000,0,19.83,서울시 영등포구 당산동4가,13
...,...,...,...,...,...,...,...
741,32831032,월세,200,65,18.18,서울시 마포구 성산동,7
742,32775788,전세,16500,0,19.83,서울시 마포구 성산동,1
743,32589981,전세,43000,0,49.00,서울시 마포구 중동,7
744,32862490,전세,43000,0,31.74,서울시 마포구 중동,5


In [152]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [153]:
data = 1

In [154]:
%whos

Variable   Type    Data/Info
----------------------------
data       int     1
